# Breeding Site Detection Model

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import time
import collections
import json

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from tqdm import tqdm
%matplotlib inline

sys.path.insert(0,os.path.abspath(".."))
from utils import label_map_util
from utils import visualization_utils as vis_util

## Processing GSV images corresponding to subdistricts

In [2]:
def process(FILES, DIR, province, district, subdist): 
    with detection_graph.as_default():
        with tf.Session(graph=detection_graph) as sess:

            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
            detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            
            for image_path in tqdm(FILES):
                image = Image.open(os.path.join(DIR, 'original', image_path))
                image_np = load_image_into_numpy_array(image)
                
                image_np_expanded = np.expand_dims(image_np, axis=0)

                (boxes, scores, classes, num) = sess.run(
                    [detection_boxes, detection_scores, detection_classes, num_detections],
                    feed_dict={image_tensor: image_np_expanded}
                )

                indexes, cls_count = count(np.squeeze(boxes),
                                           np.squeeze(classes).astype(np.int32),
                                           np.squeeze(scores),
                                           breading_index)
                
                export_json(DIR, image_path, 
                            np.squeeze(boxes), 
                            np.squeeze(classes).astype(np.int32),
                            np.squeeze(scores),
                            breading_index,
                            indexes, cls_count,
                            province, district, subdist
                           )

## Helper function to load image

In [3]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

## Count the number of detected breeding sites for each class

In [4]:
def count(boxes, classes, scores, breading_index, max_boxes_to_draw=30):
    cls_count = {}
    i, count = 0, 0
    indexes = []
    while(i < boxes.shape[0] and count < max_boxes_to_draw):
        if classes[i] in breading_index.keys():
            if scores[i] > breading_index[classes[i]]['threshold']:
                class_name = breading_index[classes[i]]['name']
                
                if class_name == 'car': 
                    i+=1
                    continue
                    
                if class_name not in cls_count:
                    cls_count[class_name] = 0
                cls_count[class_name] += 1
                
                indexes.append(i)
                count+=1
        i+=1
    return indexes, cls_count

## Export the results to Json files

In [5]:
def export_json(DIR ,image_path, boxes, classes, scores, breading_index, indexes, cls_count, 
                province, district, subdist):
    
    lat, lng, angle, date = image_path[:-4].split('_')
    data =  {
        "province": province,
        "district": district,
        "subdist": subdist,
        "image_name": image_path,
        "coordinate": [float(lat), float(lng)],
        "angle": int(angle),
        "date":{
            "month": date.split('-')[1],
            "year": date.split('-')[0]
        },
        "breading_boxes": indexes,
        "count": cls_count,
        "boxes": []
    }
    for i in range(len(classes)):
        if i == 30: break
        data["boxes"].append({
                            "bndbox": boxes[i].tolist(),
                            "score": scores[i].item(),
                            "cls_name": breading_index[classes[i]]['name'],
                            "cls_id": classes[i].item()
                        })
        

    file_name = os.path.join(DIR, 'json', image_path[:-4]+'.json')
    
    with open(file_name, 'w') as f:
        json.dump(data, f, ensure_ascii=False, sort_keys=True, indent=4, separators=(',', ': '))

## Define detection thresholds

In [6]:
breading_index ={1: {'id': 1, 'name': 'bin',         'threshold':0.5},
                 2: {'id': 2, 'name': 'bowl',        'threshold':0.5},
                 3: {'id': 3, 'name': 'bucket',      'threshold':0.5},
                 4: {'id': 4, 'name': 'car',         'threshold':0.5},
                 5: {'id': 5, 'name': 'cup',         'threshold':0.5},
                 6: {'id': 6, 'name': 'jar',         'threshold':0.5},
                 7: {'id': 7, 'name': 'pottedplant', 'threshold':0.5},
                 8: {'id': 8, 'name': 'tire',        'threshold':0.5},
                 9: {'id': 9, 'name': 'vase',        'threshold':0.5}}

## Import detection model

In [7]:
MODEL_NAME = 'dengue_inference_graph'
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph_v3.pb'
PATH_TO_LABELS = os.path.join('dengue_inference_graph', 'dengue_label_map.pbtxt')

detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

## Get the list of directories or subdistricts
That contains path_to_directory, province, district, and subdistrict 

In [8]:
directories = []

province = 'กรุงเทพมหานคร'
DIR = '../data/GSV'

districts = os.listdir(os.path.join(DIR, province))
for district in districts:
    subdists = os.listdir(os.path.join(DIR, province, district))
    for subdist in subdists:
        directories.append([DIR, province, district, subdist])

print(directories[0])

['../data/GSV', 'กรุงเทพมหานคร', 'เขตดุสิต', 'แขวงสวนจิตรลดา']


## Iterating though directories

In [9]:
last = 0
for k, directory in enumerate(directories[last:]):
    DIR, province, district, subdist = directory
    directory = os.path.join(DIR, province, district, subdist)
    
    image_path = os.path.join(directory, 'original')

    if not os.path.exists(image_path): continue

    if not os.path.exists(os.path.join(directory, 'json')):
        os.makedirs(os.path.join(directory, 'json'))

    FILES = os.listdir(image_path)
    FILES.sort()

    print('Index:',k+last,':\tDirectory:',directory, '\tTotal:',len(FILES),'Files')

    process(FILES, directory, province, district, subdist)

  0%|          | 0/1 [00:00<?, ?it/s]

Index: 0 :	Directory: ../data/GSV/กรุงเทพมหานคร/เขตดุสิต/แขวงสวนจิตรลดา 	Total: 1 Files


100%|██████████| 1/1 [00:09<00:00,  9.65s/it]
